In [1]:
import datetime
import glob
import itertools
import os

import numpy as np
import pandas as pd

##### rename the sheets of images such that there is consistency in the database filenames across campaigns
##### each campaign needs to be hardcoded and checked for appropriate syntax

In [ ]:
campaign = "ISDAC"
path = "/data/data/cpi_data/campaigns/" + campaign + "/sheets/"
for file in os.listdir(path):
    # print(file)
    filename = file[2:6] + "_" + file[6:]
    # filename=filename[:4]+'_'+filename[4:8]+'_'+filename[8:]+'.png'
    # print(filename)

    os.rename(path + file, path + filename)

In [ ]:
# OLYMPEX was a bit different in the naming scheme
campaign = "OLYMPEX"
path = "/data/data/cpi_data/campaigns/" + campaign + "/sheets/"
for file in os.listdir(path):
    # print(file)
    filename = "2015_" + file[:4] + "_" + file[5:]
    # filename=file[:4]+'_'+file[4:]
    # print(filename)

    os.rename(path + file, path + filename)

In [ ]:
# merge df's from each campaign
# no longer using this since one df got too large
dfc = []
campaigns = []
for campaign in [
    "AIRS_II",
    "ARM",
    "CRYSTAL_FACE_NASA",
    "CRYSTAL_FACE_UND",
    "ICE_L",
    "MIDCIX",
    "MPACE",
    "OLYMPEX",
]:
    df = pd.read_csv("../final_databases/no_mask/" + campaign + ".csv")
    # print(len(df))
    campaigns.append([campaign] * len(df))
    dfc.append(df)
merged = [item for items in campaigns for item in items]
dfc = pd.concat(dfc)
dfc["campaign"] = merged
cols = list(dfc)
cols.insert(1, cols.pop(cols.index("campaign")))  # put in 1st index
dfc = dfc.loc[:, cols]
dfc.to_csv("../final_databases/no_mask/all_campaigns.csv")

df_CPI = pd.read_csv("../final_databases/no_mask/all_campaigns.csv")
df_CPI.drop(columns="Unnamed: 0", inplace=True)
df_CPI

In [ ]:
dates = []
years = {
    "AIRS_II": 2003,
    "CRYSTAL_FACE_UND": 2002,
    "CRYSTAL_FACE_NASA": 2002,
    "MIDCIX": 2004,
    "ICE_L": 2007,
    "OLYMPEX": 2015,
    "MPACE": 2004,
}

for dat, camp in zip(df_CPI["filename"].str.split("_"), df_CPI["campaign"]):
    year = years[campaign]
    if camp == "OLYMPEX" and dat[0] == 11 or dat[0] == 12:
        year = 2014
    dat = str(year) + dat[0] + dat[1]
    dates.append(dat)

In [ ]:
# df_CPI['date'].str.split('-').str[0].str[4:6].max()
for row in df_CPI["date"]:
    print(row)
    pd.to_datetime(row, format="%Y%m%d-%H%M%S%f")

In [ ]:
df_CPI["date"] = dates
df_CPI["date"] = pd.to_datetime(df_CPI["date"], format="%Y%m%d-%H%M%S%f")
cols = list(df_CPI)
# move the date to head of list using index, pop and insert
cols.insert(2, cols.pop(cols.index("date")))
cols

df_CPI = df_CPI.loc[:, cols]
df_CPI

In [ ]:
df_CPI["date"].groupby([df_CPI["date"].dt.month, df_CPI["date"].dt.day]).agg("count")

In [ ]:
df_CPI = df_CPI.sort_values(by="date")

In [ ]:
g = df_CPI["date"].groupby([(df_CPI["date"].dt.day)])
df_CPI["date"].head()

In [ ]:
df_CPI.to_csv("final_databases/no_mask/all_campaigns_datesorted.csv", index=False)